<a href="https://colab.research.google.com/github/pauljainta/Machine-Learning-Project-Adaboost-/blob/main/1605022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [340]:
pip install numpy scikit-learn pandas matplotlib

In [341]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [342]:
%cd gdrive/MyDrive/dataset/

[Errno 2] No such file or directory: 'gdrive/MyDrive/dataset/'
/content/gdrive/MyDrive/dataset


In [343]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [344]:
telco_customer_dataset=pd.read_csv('telco.csv')
telco_customer_dataset.drop('customerID', inplace=True, axis=1)
# telco_customer_dataset.head(20)
# telco_customer_dataset['gender']=telco_customer_dataset['gender'].map({'Female':0,'Male':1})
# telco_customer_dataset['Partner']=telco_customer_dataset['Partner'].map({'Yes':1,'No':0})
# telco_customer_dataset.head(20)

# for i in telco_customer_dataset.columns:
#   # print(telco_customer_dataset[i].dtypes)
#   if(telco_customer_dataset[i].nunique()==2 and telco_customer_dataset[i].dtypes=='object'):
#     unique_values=telco_customer_dataset[i].unique()
#     telco_customer_dataset[i]=telco_customer_dataset[i].map({unique_values[0]:0,unique_values[1]:1})




In [345]:
def binarize_values(dataset):
  for i in dataset.columns:
  # print(telco_customer_dataset[i].dtypes)
    if(dataset[i].nunique()==2 and dataset[i].dtypes=='object'):
      unique_values=dataset[i].unique()
      dataset[i]=dataset[i].map({unique_values[0]:0,unique_values[1]:1})

  return dataset  


In [346]:
def normalize(dataset):
  for i in dataset.columns:
    if(dataset[i].nunique()!=2):
      dataset[i] = (dataset[i]-dataset[i].min())/(dataset[i].max()-dataset[i].min())
  return dataset  


In [347]:
telco_customer_dataset=binarize_values(telco_customer_dataset)
# telco_customer_dataset.head(5)

In [348]:
telco_customer_dataset['TotalCharges'] =  pd.to_numeric(telco_customer_dataset['TotalCharges'] ,errors = 'coerce')
min_occcured_value=telco_customer_dataset['TotalCharges'].value_counts().idxmin()
  # print(adult_dataset[i].unique())
telco_customer_dataset['TotalCharges']=telco_customer_dataset['TotalCharges'].replace(np.nan,min_occcured_value)

telco_customer_dataset=pd.get_dummies(telco_customer_dataset)
telco_customer_dataset=normalize(telco_customer_dataset)
# for i in telco_customer_dataset.columns:
  # if(telco_customer_dataset[i].nunique()!=2):
  #   telco_customer_dataset[i] = (telco_customer_dataset[i]-telco_customer_dataset[i].min())/(telco_customer_dataset[i].max()-telco_customer_dataset[i].min())
telco_customer_dataset.head(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,0,0,0.013889,0,0,0.115423,0.001275,0,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0
1,1,0,1,0,0.472222,1,1,0.385075,0.215867,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1
2,1,0,1,0,0.027778,1,0,0.354229,0.010310,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1
3,1,0,1,0,0.625000,0,1,0.239303,0.210241,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,0
4,0,0,1,0,0.027778,1,0,0.521891,0.015330,1,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0


In [349]:
X_telco=telco_customer_dataset.loc[:, telco_customer_dataset.columns != 'Churn']
Y_telco=telco_customer_dataset['Churn']
# X_telco=X_telco[1:1000]
# Y_telco=Y_telco[1:1000]


In [350]:
adult_dataset=pd.read_csv('adult.csv')
# adult_dataset.info()
# adult_dataset.isnull().sum()


for i in adult_dataset.columns:
  # print(adult_dataset[i].dtypes)
  # adult_dataset[i].replace('?','')
  max_occcured_value=adult_dataset[i].value_counts().idxmax()
  # print(adult_dataset[i].unique())
  adult_dataset[i]=adult_dataset[i].replace(" ?",max_occcured_value)
# adult_dataset.head(400)

adult_dataset=adult_dataset[10000:20000]

In [351]:
# for i in adult_dataset.columns:
#   # print(telco_customer_dataset[i].dtypes)
#   if(adult_dataset[i].nunique()==2 and adult_dataset[i].dtypes=='object'):
#     unique_values=adult_dataset[i].unique()
#     adult_dataset[i]=adult_dataset[i].map({unique_values[0]:0,unique_values[1]:1})

adult_dataset=binarize_values(adult_dataset)

adult_dataset=pd.get_dummies(adult_dataset)

# adult_dataset.head(4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [352]:
# for i in adult_dataset.columns:
#   if(adult_dataset[i].nunique()!=2):
#     adult_dataset[i] = (adult_dataset[i]-adult_dataset[i].min())/(adult_dataset[i].max()-adult_dataset[i].min())

adult_dataset=normalize(adult_dataset)
adult_dataset.head(5)

,age,fnlwgt,education-num,sex,capital-gain,capital-loss,hourse-per-week,Income,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,education_ 10th,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,education_ 7th-8th,education_ 9th,education_ Assoc-acdm,education_ Assoc-voc,education_ Bachelors,education_ Doctorate,education_ HS-grad,education_ Masters,education_ Preschool,education_ Prof-school,education_ Some-college,marital-status_ Divorced,marital-status_ Married-AF-spouse,marital-status_ Married-civ-spouse,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,occupation_ Adm-clerical,...,native-country_ Canada,native-country_ China,native-country_ Columbia,native-country_ Cuba,native-country_ Dominican-Republic,native-country_ Ecuador,native-country_ El-Salvador,native-country_ England,native-country_ France,native-country_ Germany,native-country_ Greece,native-country_ Guatemala,native-country_ Haiti,native-country_ Holand-Netherlands,native-country_ Honduras,native-country_ Hong,native-country_ Hungary,native-country_ India,native-country_ Iran,native-country_ Ireland,native-country_ Italy,native-country_ Jamaica,native-country_ Japan,native-country_ Laos,native-country_ Mexico,native-country_ Nicaragua,native-country_ Outlying-US(Guam-USVI-etc),native-country_ Peru,native-country_ Philippines,native-country_ Poland,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
10000,0.232877,0.073468,0.600000,0,0.0,0.0,0.500000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
10001,0.082192,0.173768,0.800000,0,0.0,0.0,0.448980,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
10002,0.506849,0.166428,0.600000,1,0.0,0.0,0.397959,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
10003,0.438356,0.136557,0.866667,1,0.0,0.0,0.704082,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
10004,0.109589,0.197733,0.533333,0,0.0,0.0,0.397959,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [353]:
X_adult=adult_dataset.loc[:, adult_dataset.columns != 'Income']
# X_adult
Y_adult=adult_dataset['Income']
Y_adult

10000    0
10001    0
10002    1
10003    1
10004    0
        ..
19995    1
19996    0
19997    0
19998    1
19999    1
Name: Income, Length: 10000, dtype: int64

In [354]:
from sklearn.model_selection import train_test_split
x_telco_train,x_telco_test,y_telco_train,y_telco_test = train_test_split(X_telco,Y_telco, test_size = 0.2)

x_adult_train,x_adult_test,y_adult_train,y_adult_test = train_test_split(X_adult,Y_adult, test_size = 0.2)

x_telco_test


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
4789,0,0,1,0,0.013889,1,1,0.525871,0.006035,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1
361,0,0,1,0,0.569444,1,0,0.801493,0.454691,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0
4331,1,0,1,0,0.986111,1,1,0.072637,0.216859,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0
418,0,0,0,1,0.305556,1,1,0.647264,0.210835,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0
960,1,1,0,0,0.097222,0,1,0.114925,0.023269,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4410,0,0,0,0,0.222222,1,0,0.760199,0.166438,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0
3491,1,0,0,0,0.611111,1,0,0.524378,0.372911,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,1,0
4280,1,0,0,1,0.888889,1,0,0.014428,0.144848,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0
461,0,0,0,0,0.333333,1,0,0.509453,0.184081,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0


In [355]:
def hypothesis(theta,x):
    z = 0
    for i in range(len(theta)):
        xi = x[i]
        z += np.dot(xi,theta[i].transpose())
    # if(np.isnan(np.tanh(z))):
    #   return 0    
    return np.tanh(z)

In [356]:
def cost_function_derivativetanh(theta,j,m,alpha,x_train,y_train):
    sumErrors = 0
    for i in range(m):
        xi = x_train.iloc[i]
        hi = hypothesis(theta,xi)
        yt = y_train.iloc[i]
        hit = np.subtract((float)(hi),float(yt))
        error = np.dot(hit,x_train.iloc[i][j])
        sumErrors += error
    # m = len(y_test)
    constant = float(alpha)/float(m)
    j = constant * sumErrors
    return j

In [357]:
def gradient_descenttanh(theta,m,alpha,x_train,y_train):
    new_theta = []
    for j in range(len(theta)):
        new_theta_value = theta[j] - cost_function_derivativetanh(theta,j,m,alpha,x_train,y_train)
        new_theta.append(new_theta_value)
    return new_theta

In [358]:
def logistic_regression(alpha,theta,num_iters,x_train,x_test,y_train,y_test):
    m = len(y_train)
    for x in range(num_iters):
        new_theta = gradient_descenttanh(theta,m,alpha,x_train,y_train)
        theta = new_theta
        # if x % 100 == 0:
        #     cost_functiontanh(theta,m)
        #     print ('theta ', theta)	
        #     print ('cost is ', cost_functiontanh(theta,m))
    score = 0
    length = len(x_test)
    for i in range(length):
        prediction = round(hypothesis(theta,x_test.iloc[i]))
        answer = y_test.iloc[i]
        if prediction == answer:
            score += 1
    my_score = float(score) / float(length)
    print ('Your score with tanh: ', my_score)

In [ ]:
initial_theta = np.zeros(x_telco_test.iloc[1].shape)
alpha = 0.1
iterations = 10
# x_telco_test.shape
logistic_regression(alpha,initial_theta,iterations,x_telco_train,x_telco_test,y_telco_train,y_telco_test)